In [26]:
# Importar librerías
import os
import pandas as pd
import numpy as np
import requests
import boto3
import yaml

In [34]:
# Cargar configuración desde el archivo config.yaml
with open('./config.yaml', 'r') as f:
    config = yaml.safe_load(f)


# Obtener el token del archivo de configuración
token = config.get('token', None)
#token = 'b6776d08cb636cb8cab098f7b6ba07ab94ef24d130a73e9bb911d7605cc41c83'
#token = 'eef03bd2dd8aa693023733eb59b5b2b33d2be9f38b8d7b228ace1b0f063ee6d7'

def bmx_indown(ind, start, finish, token):
    # URL del endpoint para obtener el tipo de cambio
    url = 'https://www.banxico.org.mx/SieAPIRest/service/v1/series/'+ind+'/datos/'+start+'/'+finish

    # Agregar tu API key a los headers de la solicitud
    headers = {'Bmx-Token':token}

    # Realizar la solicitud GET
    response = requests.get(url, headers=headers)

    status = response.status_code

    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Convertir la respuesta a formato JSON
        data = response.json()

        # Extraer el tipo de cambio
        indicador = data['bmx']['series'][0]['datos']
        df = pd.DataFrame(indicador)
        df['dato'] = df['dato'].apply(lambda x: float(x))
        df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')
        df = df.set_index('fecha')

        return df
    else:
        # Si la solicitud falla, imprimir el código de estado
        print("Error al obtener el indicador. Código de estado:", response.status_code)
        return None


In [35]:
# Ejemplo de uso
tdcd_bmx = bmx_indown('SF63528', '2023-01-01', '2023-12-31', token)
tiie_bmx = bmx_indown('SF43783', '2023-01-01', '2023-12-31', token)
sp1_bmx = bmx_indown('SP1', '2023-01-01', '2023-12-31', token)


In [21]:
# Ejemplo de cómo podrías llenar valores faltantes para cada DataFrame
for df in [tdcd_bmx, tiie_bmx, sp1_bmx]:
    # Eliminar valores faltantes
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
  

In [22]:
# Guardar tdcd_bmx a CSV
tdcd_bmx.to_csv('./data/tdcd_bmx.csv', index=True)

# Guardar tiie_bmx a CSV
tiie_bmx.to_csv('./data/tiie_bmx.csv', index=True)

# Guardar sp1_bmx a CSV
sp1_bmx.to_csv('./data/sp1_bmx.csv', index=True)


In [25]:
# Upload los archivos a S3
s3.upload_file(Filename="./data/sp1_bmx.csv", Bucket="itam-analytics-carlos", Key="raw/inflacion/sp1_bmx.csv")
s3.upload_file(Filename="./data/tdcd_bmx.csv", Bucket="itam-analytics-carlos", Key="raw/tipo_de_cambio/tdcd_bmx.csv")
s3.upload_file(Filename="./data/tiie_bmx.csv", Bucket="itam-analytics-carlos", Key="raw/tasa_de_interes/tiie_bmx.csv")